In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context(context="paper")
sns.set_style(style="whitegrid")

In [2]:
def load_pos_data(path: str) -> pd.DataFrame:
    
    data = pd.read_csv(path, index_col=[2, 1])["position"].unstack().fillna(0)
    data.index = pd.to_datetime(data.index)

    return data

def load_price_data(*, path: str, **kwargs) -> pd.DataFrame:

    data = pd.read_csv(path).pivot_table(**kwargs)
    data.index = pd.to_datetime(data.index.map(lambda x: str(x)))

    return data

options = load_price_data(path="options.csv", values="收盘价", index="date", columns="合约代码")
futures = load_price_data(path="futures.csv", values="dominant_contract_price", index="date", columns="dominant_contract")
price = pd.merge(left=futures, right=options, left_index=True, right_index=True, how="inner")
price

,au2006,au2012,au2106,au2112,au2202,au2206,au2212,au2302,au2304,au2306,...,au2508P672,au2508P680,au2508P688,au2508P696,au2508P704,au2508P712,au2508P720,au2508P728,au2508P736,au2508P744
date,,,,,,,,,,,,,,,,,,,,,
2019-12-20,338.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-23,339.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-24,340.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-25,343.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-26,343.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.66,65.34,71.34,77.44,83.82,90.36,97.04,103.94,110.88,118.06
2024-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.56,61.24,67.16,73.26,79.62,86.16,92.84,99.74,106.74,113.90
2024-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,60.62,66.30,72.14,78.20,84.48,90.88,97.54,104.24,111.18,118.20


In [3]:
class Backtest():

    def __init__(self, equity: int, price: pd.DataFrame, benchmark: pd.Series) -> None:

        self.equity = equity
        self.price = price
        self.benchmark = benchmark

        print("Backtest Initialized ---------------\n## %s ~ %s\n## %d time points\n"
              % (self.price.index[0].strftime("%Y-%m-%d"), 
                 self.price.index[-1].strftime("%Y-%m-%d"), 
                 len(self.price.index)))
    
    def run(self, position: pd.DataFrame, stop_profit: float=None) -> dict:

        common_index = self.price.index.intersection(position.index)
        common_columns = self.price.columns.intersection(position.columns)
        price = self.price[common_columns].loc[common_index, :]
        pos = position[common_columns].loc[common_index, :]

        print("## %s ~ %s: " % (common_index[0].strftime("%Y-%m-%d"), common_index[-1].strftime("%Y-%m-%d")), end="")
        
        pos_val = (price * pos)
        pro_and_loss = ((price * pos.shift(1)) - pos_val.shift(1)).fillna(0)
        self.pl = pro_and_loss.cumsum()
        self.unit_value = (self.pl.sum(axis=1) + self.equity) / self.equity

        if stop_profit and self.unit_value.max() >= 1 + stop_profit:
            self.stop_profit = (self.unit_value >= 1 + stop_profit).idxmax()
        else:
            self.stop_profit = None
        
        unit_value = self.unit_value[self.unit_value.index <= self.stop_profit] if self.stop_profit else self.unit_value

        result = {}

        rev_tsp = pd.Timedelta(days=365) / (unit_value.index[-1] - unit_value.index[0])
        result["tot_ret"] = unit_value.iloc[-1] / unit_value.iloc[0] - 1
        result["yrl_ret"] = (result["tot_ret"] + 1) ** rev_tsp - 1
        result["yrl_vol"] = (unit_value.diff(1) / unit_value.shift(1)).dropna().std() * np.sqrt(252)

        benchmark = self.benchmark[unit_value.index]
        result["ex_tot_ret"] = result["tot_ret"] - (benchmark.iloc[-1] / benchmark.iloc[0] - 1)
        result["ex_yrl_ret"] = (result["ex_tot_ret"] + 1) ** rev_tsp - 1
        result["ex_yrl_vol"] = (unit_value.diff(1) / unit_value.shift(1) - benchmark.diff(1) / benchmark.shift(1)) \
                                .dropna().std() * np.sqrt(252)

        result["max_draw"] = ((unit_value.cummax() - unit_value) / unit_value.cummax()).max()

        result["stp_pft"] = True if self.stop_profit else False

        print("total_return: %.2f%%, stop_profit: %s" % 
              (result["tot_ret"] * 100, str(result["stp_pft"])))

        return result

    def plot(self, *, savefig: bool=True, savepath: str=None, **kwargs) -> None:

        benchmark = self.benchmark[self.unit_value.index]
        benchmark = benchmark / benchmark.iloc[0]

        plt.figure(figsize=(12, 6))
        plt.plot(self.unit_value, label=(kwargs["label"] if "label" in kwargs.keys() else None))
        plt.plot(benchmark, color="orange", label=(kwargs["bench_label"] if "bench_label" in kwargs.keys() else None))
        if self.stop_profit:
            plt.axvline(self.stop_profit, color="red", linestyle="--")
            plt.text(self.stop_profit + pd.Timedelta(days=len(self.unit_value.index)/100), 1, 
                     "stop profit", color="red", fontdict={"fontsize": "large"})
        plt.title(label=(kwargs["title"] if "title" in kwargs.keys() else None))
        plt.legend()

        if savefig:
            if savepath: plt.savefig(savepath)
            else: plt.savefig("backtest.png")
        plt.close()

In [4]:
equity = 10000000
multiplier = 1000
stop_profit = 0.2

future_list = list(filter(lambda x: "futures" in x, os.listdir("position")))
option_list = list(filter(lambda x: "option" in x, os.listdir("position")))

spot = pd.read_csv("futures.csv", index_col=-1)["spot_price"]
spot.index = pd.to_datetime(spot.index.map(lambda x: str(x)))

backtest = Backtest(equity=equity, price=price, benchmark=spot)

result = pd.DataFrame()

for op, ft in zip(option_list, future_list):

    option = load_pos_data("position/%s" % op) * multiplier
    future = load_pos_data("position/%s" % ft) * multiplier
    pos = pd.merge(left=future, right=option, left_index=True, right_index=True, how="inner")

    result = pd.concat([result, pd.DataFrame(backtest.run(position=pos, stop_profit=stop_profit), index=[pos.index[0]])])
    backtest.plot(savefig=True, savepath="plots/%s" % ft[8: -4], 
                  label="unit_value", bench_label="gold", title=ft[8: -4])

Backtest Initialized ---------------
## 2019-12-20 ~ 2024-12-17
## 1209 time points

## 2020-01-02 ~ 2021-12-31: total_return: 24.92%, stop_profit: True
## 2020-01-08 ~ 2022-01-07: total_return: 22.92%, stop_profit: True
## 2020-01-15 ~ 2022-01-14: total_return: 20.19%, stop_profit: True
## 2020-01-22 ~ 2022-01-21: total_return: 24.69%, stop_profit: True
## 2020-02-03 ~ 2022-01-28: total_return: 22.58%, stop_profit: True
## 2020-02-05 ~ 2022-01-28: total_return: 25.72%, stop_profit: True
## 2020-02-12 ~ 2022-02-11: total_return: 25.86%, stop_profit: True
## 2020-02-19 ~ 2022-02-18: total_return: 20.79%, stop_profit: True
## 2020-02-26 ~ 2022-02-25: total_return: 20.20%, stop_profit: True
## 2020-03-04 ~ 2022-03-03: total_return: 20.78%, stop_profit: True
## 2020-03-11 ~ 2022-03-10: total_return: 24.10%, stop_profit: True
## 2020-03-18 ~ 2022-03-17: total_return: 20.68%, stop_profit: True
## 2020-03-25 ~ 2022-03-24: total_return: 23.42%, stop_profit: True
## 2020-04-01 ~ 2022-03-31: tot

In [5]:
result

,tot_ret,yrl_ret,yrl_vol,ex_tot_ret,ex_yrl_ret,ex_yrl_vol,max_draw,stp_pft
2020-01-02,0.249200,0.480436,0.186628,0.000662,0.001168,0.103731,0.128233,True
2020-01-08,0.229220,0.454656,0.172116,0.033142,0.060995,0.104101,0.101087,True
2020-01-15,0.201896,0.426397,0.167748,0.014315,0.027829,0.099677,0.109137,True
2020-01-22,0.246862,0.538244,0.186274,0.020203,0.039813,0.104335,0.111772,True
2020-02-03,0.225756,0.528923,0.181028,0.029141,0.061743,0.101582,0.102411,True
...,...,...,...,...,...,...,...,...
2021-12-01,0.209600,0.189626,0.105709,0.081687,0.074280,0.067047,0.075655,True
2021-12-08,0.200870,0.218558,0.107951,0.091913,0.099609,0.067771,0.074463,True
2021-12-15,0.201496,0.225117,0.108351,0.099909,0.111074,0.067889,0.075033,True
2021-12-22,0.211284,0.202738,0.106613,0.089341,0.085903,0.067623,0.076847,True


In [6]:
result.to_csv("backtest_result.csv")